In [178]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from pathlib import Path
import pint
import pint_pandas
import sys


In [179]:
sys.path.append(str(Path.cwd().parent))

In [180]:
from src.utils import load_pint_data, PREFERRED_UNITS
ureg = pint.get_application_registry()

In [181]:
ROOT = Path().cwd().parent
PARAMETER_SPACE = "01"
DATA_TYPE = "Training"
param_file_training = ROOT / "data" / PARAMETER_SPACE / "training_samples.csv"
assert param_file_training.exists()
param_file_test = ROOT / "data" / PARAMETER_SPACE / "test_samples.csv"
assert param_file_test.exists()

zero_crossing_file_training = ROOT / "data" / PARAMETER_SPACE / "Exports" / "Training_zero_crossings.npy"
assert zero_crossing_file_training.exists()
zero_crossing_file_test = ROOT / "data" / PARAMETER_SPACE / "Exports" / "Test_zero_crossings.npy"
assert zero_crossing_file_test.exists()

In [182]:
parameters_training = load_pint_data(param_file_training)
parameters_test     = load_pint_data(param_file_test)
zero_crossings_training = np.load(zero_crossing_file_training)
zero_crossings_test = np.load(zero_crossing_file_test)
assert len(parameters_training) == len(zero_crossings_training)
assert len(zero_crossings_test) == len(zero_crossings_test)

In [183]:
for col in parameters_training.columns:
    if parameters_training[col].pint.dimensionality in PREFERRED_UNITS:
        unit = PREFERRED_UNITS[parameters_training[col].pint.dimensionality]
        parameters_training[col] = parameters_training[col].pint.to(unit)   
        parameters_test[col] = parameters_test[col].pint.to(unit) 
          
parameters_training['zero_crossings'] = zero_crossings_training
parameters_test['zero_crossings'] = zero_crossings_test

In [ ]:
is_3d_plot = False
if PARAMETER_SPACE == "01":
    is_3d_plot = True
    entropy_number_training = np.load(ROOT / "data" / PARAMETER_SPACE / "TrainingMapped/s100_100_100_b0_4000_0_5000_-4000_0/Exports/entropy_gen_number.npy")[:, -1]
    entropy_number_test = np.load(ROOT / "data" / PARAMETER_SPACE / "TestMapped/s100_100_100_b0_4000_0_5000_-4000_0/Exports/Test_entropy_gen_number.npy")[:, -1]
    parameters_training['entropy_gen_number'] = entropy_number_training * ureg.dimensionless
    parameters_test['entropy_gen_number'] = entropy_number_test * ureg.dimensionless
    log_values_train = np.log10(parameters_training['host_k'].pint.magnitude) 
    log_values_test = np.log10(parameters_test['host_k'].pint.magnitude) 
    
    parameters_training['log10_host_k'] = pd.Series(log_values_train, dtype=pint_pandas.PintType('m**2'))
    parameters_test['log10_host_k'] = pd.Series(log_values_test, dtype=pint_pandas.PintType('m**2'))
    
    parameters_training = parameters_training.pint.dequantify()
    parameters_test = parameters_test.pint.dequantify()
    
    indices_to_highlight_training = [64, 9, 27, 22, 60, 149, 70,213,
                            73, 90, 238, 118, 50, 80, 278, 222, 14, 235,
                            40, 209, 212, 220, 227, 267, 239, 134, 268]
    
    indices_to_highlight_test = [0, 1, 3, 12, 20, 22, 24, 26, 27, 29, 28, 33,]
    
    x_key = dict(key = parameters_training.columns[1])
    y_key = dict(key = parameters_training.columns[-1]) 
    z_key = dict(key = parameters_training.columns[-2]) 
elif PARAMETER_SPACE == "03":
    parameters_training = parameters_training.pint.dequantify()
    parameters_test = parameters_test.pint.dequantify()
    parameters_training = parameters_training[:100]
    parameters_test = parameters_test[:20]
    x_key = dict(key = parameters_training.columns[0])
    y_key = dict(key = parameters_training.columns[1])
    indices_to_highlight_training = [] 
    indices_to_highlight_test = [] 
elif PARAMETER_SPACE == "07":
    entropy_number_training = np.load(ROOT / "data" / PARAMETER_SPACE / "Exports" / "Training_entropy_gen_number.npy")[:, -1]
    entropy_number_test = np.load(ROOT / "data" / PARAMETER_SPACE / "Exports" / "Test_entropy_gen_number.npy")[:, -1]
    parameters_training['entropy_gen_number'] = entropy_number_training * ureg.dimensionless
    parameters_test['entropy_gen_number'] = entropy_number_test * ureg.dimensionless
    parameters_training = parameters_training.pint.dequantify()
    parameters_test = parameters_test.pint.dequantify()
    x_key = dict(key = parameters_training.columns[0])
    y_key = dict(key = parameters_training.columns[-1])
    indices_to_highlight_training = [] 
    indices_to_highlight_test = [] 
else:
    parameters_training = parameters_training.pint.dequantify()
    parameters_test = parameters_test.pint.dequantify()
    x_key = dict(key = parameters_training.columns[0])
    y_key = dict(key = parameters_training.columns[1])
    indices_to_highlight_training = [] 
    indices_to_highlight_test = [] 
    

/Users/thomassimader/Library/CloudStorage/OneDrive-geomeconGmbH/PhD/NIRB/.venv/lib/python3.12/site-packages/pandas/core/construction.py:632: UnitStrippedWarning:

The unit of the quantity is stripped when downcasting to ndarray.

/Users/thomassimader/Library/CloudStorage/OneDrive-geomeconGmbH/PhD/NIRB/.venv/lib/python3.12/site-packages/pandas/core/construction.py:632: UnitStrippedWarning:

The unit of the quantity is stripped when downcasting to ndarray.



In [185]:
# Stack the custom data: [sample_idx, zero_crossings]
custom_data_training = np.column_stack([
    parameters_training.index.to_numpy(),
    parameters_training['zero_crossings'].to_numpy()[:, 0]
])

custom_data_test = np.column_stack([
    parameters_test.index.to_numpy(),
    parameters_test['zero_crossings'].to_numpy()[:, 0]
])

vmin = min(parameters_training['zero_crossings'].values.min(), parameters_test['zero_crossings'].values.min())
vmax = max(parameters_training['zero_crossings'].values.max(), parameters_test['zero_crossings'].values.max())
cmap = 'viridis' #'portland'

# All Training Samples
fig = go.Figure(data=go.Scatter(
    x=parameters_training[x_key.get('key')],
    y=parameters_training[y_key.get('key')],
    mode='markers',
    text = parameters_training.index.to_numpy(),
    name = 'Training Samples',
    marker=dict(
        size=8,
        color=parameters_training['zero_crossings'].to_numpy()[:, 0],
        colorscale=cmap,
        showscale=True,
        opacity=0.9,
        cmin=vmin,
        cmax=vmax,
    ),
    customdata=custom_data_training,
    hovertemplate = (
        'Training Idx: %{customdata[0]:03d}<br>' +
        f'{x_key.get('key')}: %{{x:.2f}}<br>' +
        f'{y_key.get('key')}: %{{y:.2f}}<br>' +
        'zero_crossings: %{customdata[1]:d}<extra></extra>'
    )
))

# All Test Samples
fig.add_trace(go.Scatter(
    x=parameters_test[x_key.get('key')],
    y=parameters_test[y_key.get('key')],
    mode='markers',
    text = parameters_test.index.to_numpy(),
    name = 'Test Samples',
    marker=dict(
        size=8,
        color= 'rgba(0,0,0,0)',  # transparent fill
        line=dict(
            colorscale=cmap,
            color=parameters_test['zero_crossings'].to_numpy()[:, 0],
            width=3,
            cmin=vmin,
            cmax=vmax,
        ),
        showscale=False,
        opacity=0.9,

    ),
    customdata=custom_data_test,
    hovertemplate=(
        'Test Idx: %{customdata[0]:03d}<br>' +
        f'{x_key.get('key')}: %{{x:.2f}}<br>' +
        f'{y_key.get('key')}: %{{y:.2f}}<br>' +
        'zero_crossings: %{customdata[1]:d}<extra></extra>'
    )
))


highlight_mask = parameters_training.index.isin(indices_to_highlight_training)
# Highlighted points
fig.add_trace(go.Scatter(
    x=parameters_training.loc[highlight_mask, x_key.get('key')],
    y=parameters_training.loc[highlight_mask, y_key.get('key')],
    mode='markers',
    name = 'Highlighted Training Samples',
    marker=dict(
        showscale = False,
        size=14,
        color='rgba(0,0,0,0)',  # Transparent fill
        line=dict(color='red', width=2)  # Red outline
    ),
    hoverinfo = 'skip'
))


highlight_mask = parameters_test.index.isin(indices_to_highlight_test)
# Highlighted points
fig.add_trace(go.Scatter(
    x=parameters_test.loc[highlight_mask, x_key.get('key')],
    y=parameters_test.loc[highlight_mask, y_key.get('key')],
    mode='markers',
    name = 'Critical Test Samples',
    marker=dict(
        showscale = False,
        size=14,
        color='rgba(0,0,0,0)',  # Transparent fill
        line=dict(color='magenta', width=2)  # Red outline
    ),
    hoverinfo = 'skip'
))

# All Training Samples
fig.add_trace(go.Scatter(
    x=parameters_training[x_key.get('key')],
    y=parameters_training[y_key.get('key')],
    mode='text',
    textposition='top center',
    text = parameters_training.index.to_numpy(),
    name = 'Training Samples Idx Numbers',
    hovertemplate='skip',
    visible='legendonly' 
))

fig.add_trace(go.Scatter(
    x=parameters_test[x_key.get('key')],
    y=parameters_test[y_key.get('key')],
    mode='text',
    textposition='top center',
    text = parameters_test.index.to_numpy(),
    name = 'Test Samples Idx Numbers',
    hovertemplate='skip',
    visible='legendonly' 
))

fig.update_layout(
    title=f'Parameter Space {PARAMETER_SPACE} - Number of Convection Cells',
    xaxis_title=' - '.join(x_key.get('key')),
    yaxis_title=' - '.join(y_key.get('key')),
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=-0.5,
        xanchor="left",
        x=0,
        traceorder="normal",
        itemwidth=70  # control spacing
    )
)

fig.show()
fig.write_html(ROOT / "data" / PARAMETER_SPACE / "Exports" / f"Zero_Crossing_{PARAMETER_SPACE}.html")


In [197]:
if is_3d_plot:

    # Stack the custom data: [sample_idx, zero_crossings]
    custom_data_training = np.column_stack([
        parameters_training.index.to_numpy(),
        parameters_training['zero_crossings'].to_numpy()[:, 0]
    ])

    custom_data_test = np.column_stack([
        parameters_test.index.to_numpy(),
        parameters_test['zero_crossings'].to_numpy()[:, 0]
    ])

    vmin = min(parameters_training['zero_crossings'].values.min(), parameters_test['zero_crossings'].values.min())
    vmax = max(parameters_training['zero_crossings'].values.max(), parameters_test['zero_crossings'].values.max())
    cmap = 'viridis' #'portland'

    # All Training Samples
    fig = go.Figure(data=go.Scatter3d(
        x=parameters_training[x_key.get('key')].values,
        y=parameters_training[y_key.get('key')].values,
        z=parameters_training[z_key.get('key')].values,
        mode='markers',
        text = parameters_training.index.to_numpy(),
        name = 'Training Samples',
        marker=dict(
            size=5,
            color=parameters_training['zero_crossings'].to_numpy()[:, 0],
            colorscale=cmap,
            showscale=True,
            opacity=0.9,
            cmin=vmin,
            cmax=vmax,
        ),
        customdata=custom_data_training,
        hovertemplate = (
            'Training Idx: %{customdata[0]:03d}<br>' +
            f'{x_key.get('key')}: %{{x:.2f}}<br>' +
            f'{y_key.get('key')}: %{{y:.2f}}<br>' +
            'zero_crossings: %{customdata[1]:d}<extra></extra>'
        )
    ))

    # All Test Samples
    fig.add_trace(go.Scatter3d(
        x=parameters_test[x_key.get('key')],
        y=parameters_test[y_key.get('key')],
        z=parameters_test[z_key.get('key')],
        mode='markers',
        text = parameters_test.index.to_numpy(),
        name = 'Test Samples',
        marker=dict(
            size=5,
            color= 'rgba(0,0,0,0)',  # transparent fill
            line=dict(
                colorscale=cmap,
                color=parameters_test['zero_crossings'].to_numpy()[:, 0],
                width=4,
                cmin=vmin,
                cmax=vmax,
            ),
            showscale=False,
            opacity=0.9,

        ),
        customdata=custom_data_test,
        hovertemplate=(
            'Test Idx: %{customdata[0]:03d}<br>' +
            f'{x_key.get('key')}: %{{x:.2f}}<br>' +
            f'{y_key.get('key')}: %{{y:.2f}}<br>' +
            'zero_crossings: %{customdata[1]:d}<extra></extra>'
        )
    ))

    # # All Training Samples
    fig.add_trace(go.Scatter3d(
        x=parameters_training[x_key.get('key')],
        y=parameters_training[y_key.get('key')],
        z=parameters_training[z_key.get('key')],
        mode='text',
        textposition='top center',
        text = parameters_training.index.to_numpy(),
        name = 'Training Samples Idx Numbers',
        hovertemplate='skip',
        visible='legendonly' 
    ))

    fig.add_trace(go.Scatter3d(
        x=parameters_test[x_key.get('key')],
        y=parameters_test[y_key.get('key')],
        z=parameters_test[z_key.get('key')],
        mode='text',
        textposition='top center',
        text = parameters_test.index.to_numpy(),
        name = 'Test Samples Idx Numbers',
        hovertemplate='skip',
        visible='legendonly' 
    ))

    fig.update_layout(
        title=f'Parameter Space {PARAMETER_SPACE} - Number of Convection Cells',
        scene=dict(
            xaxis=dict(title = dict(text = '-'.join(x_key.get('key')))),
            yaxis=dict(title = dict(text = '-'.join(y_key.get('key')))),
            zaxis=dict(title = dict(text = '-'.join(z_key.get('key')))),
        ),
        legend=dict(
        orientation="h",
        yanchor="bottom",
        y=-0.5,
        xanchor="left",
        x=0,
        )
    )

    fig.show()
    fig.write_html(ROOT / "data" / PARAMETER_SPACE / "Exports" / f"Zero_Crossing_{PARAMETER_SPACE}_3D.html")
